# POKEMON DATABASE - DATA ANALYSIS

> Neste notebook, vamos explorar e analisar uma base de dados completa de Pokémon, extraída de fontes públicas e enriquecida com informações adicionais, como atributos de batalha, habilidades, fraquezas e locais de captura.

O objetivo deste trabalho é aplicar técnicas de análise de dados para responder perguntas relevantes, identificar padrões, tendências e insights que possam ser úteis para jogadores, pesquisadores ou entusiastas do universo Pokémon.

Ao longo do notebook, você encontrará:
- Extração, limpeza e preparação dos dados 
- Dicionário de dados
- Análise de qualidade e exploratória dos dados
- Visualizações para facilitar a compreensão dos dados
- Descoberta de relações entre tipos, habilidades, fraquezas e desempenho
- Insights sobre distribuição, raridade e características marcantes dos Pokémon

## Extração, Limpeza e Preparação dos Dados

O processo de construção da base de dados utilizada nesta análise envolveu diversas etapas de extração, enriquecimento e preparação dos dados, conforme descrito abaixo:

- **Extração Inicial:**  
  A base principal de Pokémon foi extraída do site oficial [Pokédex Pokémon](https://www.pokemon.com/br/pokedex), contendo informações básicas de cada espécie.

- **Enriquecimento com Localização de Captura:**  
  Utilizou-se a [PokeAPI](https://pokeapi.co/) para complementar a base, adicionando as localizações de captura de cada Pokémon, tornando a análise mais rica e detalhada.

- **Codificação de Dados Categóricos:**  
  Para lidar com atributos que possuem múltiplos valores (como habilidades, tipos, fraquezas e locais de captura), foi aplicada a técnica de One Hot Encoding, facilitando a análise e a modelagem dos dados.

- **Resgate de Dados de Status:**  
  Os dados de status dos Pokémon (HP, Attack, Defense, Sp. Attack, Sp. Defense e Speed) foram extraídos do site [Pokémon Database](https://pokemondb.net/pokedex/all) utilizando a biblioteca BeautifulSoup para leitura do HTML.  
  - Para Pokémon com diferentes formas, foi considerado o status da forma padrão/normal.

Essas etapas garantem uma base de dados consolidada, limpa e pronta para análises mais profundas e visualizações

In [6]:
import json
import pandas as pd

# Ler o JSON extraindo do site oficial da pokedex
with open('./assets/pokedex-db.json', "r", encoding="utf-8") as f:
    pokedex_data = json.load(f)

# Converter para DataFrame
df = pd.DataFrame(pokedex_data)

print(f"Tamanho do DataFrame: {df.shape}")
print(f"Colunas do DataFrame: {df.columns.tolist()}")

Tamanho do DataFrame: (1025, 14)
Colunas do DataFrame: ['abilities', 'detailPageURL', 'weight', 'weakness', 'number', 'height', 'collectibles_slug', 'featured', 'slug', 'name', 'ThumbnailAltText', 'ThumbnailImage', 'id', 'type']


In [7]:

# Removendo colunas desnecessárias se existirem
if 'ThumbnailAltText' in df.columns:
    df.drop(columns=['ThumbnailAltText'], inplace=True)
if 'detailPageURL' in df.columns:
    df.drop(columns=['detailPageURL'], inplace=True)
if 'collectibles_slug' in df.columns:
    df.drop(columns=['collectibles_slug'], inplace=True)
if 'featured' in df.columns:
    df.drop(columns=['featured'], inplace=True)
if 'slug' in df.columns:
    df.drop(columns=['slug'], inplace=True)
if 'ThumbnailImage' in df.columns:
    df.drop(columns=['ThumbnailImage'], inplace=True)
     
print(f"Tamanho do DataFrame após remoção de colunas: {df.shape}\n\n")
print(df.head(5))
        

Tamanho do DataFrame após remoção de colunas: (1025, 8)


    abilities  weight                      weakness number  height  \
0  [Overgrow]     6.9  [Fire, Ice, Flying, Psychic]   0001     0.7   
1  [Overgrow]    13.0  [Fire, Ice, Flying, Psychic]   0002     1.0   
2  [Overgrow]   100.0  [Fire, Ice, Flying, Psychic]   0003     2.0   
3     [Blaze]     8.5         [Water, Ground, Rock]   0004     0.6   
4     [Blaze]    19.0         [Water, Ground, Rock]   0005     1.1   

         name  id             type  
0   Bulbasaur   1  [grass, poison]  
1     Ivysaur   2  [grass, poison]  
2    Venusaur   3  [grass, poison]  
3  Charmander   4           [fire]  
4  Charmeleon   5           [fire]  


In [8]:
from bs4 import BeautifulSoup
import json

# Abra e leia o arquivo HTML
with open('./assets/pokemondb-net.html', 'r', encoding='utf-8') as f:
    html = f.read()

soup = BeautifulSoup(html, 'html.parser')

# Encontre a tabela com id="pokedex"
table = soup.find('table', {'id': 'pokedex'})

# Pegue o cabeçalho da tabela
headers = [th.get_text(strip=True) for th in table.find('thead').find_all('th')]

# Descubra o índice da coluna 'Type'
type_index = headers.index('Type')

# Remova 'Type' dos headers
headers.pop(type_index)

# Extraia os dados das linhas
data = []
for row in table.find('tbody').find_all('tr'):
    cols = row.find_all('td')
    if not cols:
        continue

    # Extrai os textos das colunas, exceto 'Type'
    col_texts = [td.get_text(strip=True) for td in cols]
    col_texts.pop(type_index)

    # Tratamento especial para a coluna 'Name'
    name_td = cols[headers.index('Name')]
    name_a = name_td.find('a')
    name = name_a.get_text(strip=True) if name_a else name_td.get_text(strip=True)
    form_small = name_td.find('small')
    form = form_small.get_text(strip=True) if form_small else None

    # Substitui o valor da coluna 'Name' pelo nome extraído
    col_texts[headers.index('Name')] = name

    # Cria o dicionário
    item = dict(zip(headers, col_texts))
    if form:
        item['form'] = form
    data.append(item)
    
# Remover os pokemons repetidos mantendo apenas o que primeiro aparece na lista
unique_data = []
seen_ids = set()
for item in data:
    poke_id = item.get("#")
    if poke_id and poke_id not in seen_ids:
        unique_data.append(item)
        seen_ids.add(poke_id)
 
# Salvar em um arquivo JSON
with open('./outputs/pokemon-stats-db.json', 'w', encoding='utf-8') as f:
    json.dump(unique_data, f, ensure_ascii=False, indent=4)

# Concatenar os dados de 'attack', 'defense', 'hp', 'sp_atk', 'sp_def', 'speed' ao DataFrame df comparando o "id" do df com o "#" do unique_data
for item in unique_data:
    poke_id = item.get("#")
    if poke_id and poke_id.isdigit():
        poke_id = int(poke_id)

        df.loc[df['id'] == poke_id, ['attack', 'defense', 'hp', 'sp_atk', 'sp_def', 'speed', 'total']] = [
            int(item.get('Attack')),
            int(item.get('Defense')),
            int(item.get('HP')),
            int(item.get('Sp. Atk')),
            int(item.get('Sp. Def')),
            int(item.get('Speed')),
            int(item.get('Total'))
        ]
        
    
# Ordenando colunas
df = df[['id', 'number', 'name', 'type', 'height', 'weight', 'attack', 'defense', 'hp', 'sp_atk', 'sp_def', 'speed', 'total', 'abilities', 'weakness']]
         
print(f"Tamanho do DataFrame após atualização: {df.shape}\n\n")
print(df.head())  # Exibir as primeiras linhas do DataFrame atualizado

Tamanho do DataFrame após atualização: (1025, 15)


   id number        name             type  height  weight  attack  defense  \
0   1   0001   Bulbasaur  [grass, poison]     0.7     6.9    49.0     49.0   
1   2   0002     Ivysaur  [grass, poison]     1.0    13.0    62.0     63.0   
2   3   0003    Venusaur  [grass, poison]     2.0   100.0    82.0     83.0   
3   4   0004  Charmander           [fire]     0.6     8.5    52.0     43.0   
4   5   0005  Charmeleon           [fire]     1.1    19.0    64.0     58.0   

     hp  sp_atk  sp_def  speed  total   abilities  \
0  45.0    65.0    65.0   45.0  318.0  [Overgrow]   
1  60.0    80.0    80.0   60.0  405.0  [Overgrow]   
2  80.0   100.0   100.0   80.0  525.0  [Overgrow]   
3  39.0    60.0    50.0   65.0  309.0     [Blaze]   
4  58.0    80.0    65.0   80.0  405.0     [Blaze]   

                       weakness  
0  [Fire, Ice, Flying, Psychic]  
1  [Fire, Ice, Flying, Psychic]  
2  [Fire, Ice, Flying, Psychic]  
3         [Water, Ground

In [9]:
import requests
import time
import os

# Verificando Cache
if os.path.exists('./outputs/pokemon_capture_locations.json'):
    df_capture_locations = pd.read_json('./outputs/pokemon_capture_locations.json', orient='records')
    df = df.merge(df_capture_locations, on='id', how='outer')
    print("Dados de localizações de captura carregados do cache.")
    
# Resgatando Localizações de Captura 
else:
    def get_capture_locations(number):
        url = f"https://pokeapi.co/api/v2/pokemon/{number}/encounters"
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                data = response.json()
                
                # Extrai o nome dos locais de captura
                locations = [loc['location_area']['name'] for loc in data]
                return locations
            else:
                return []
        except Exception as e:
            return []
    
    # Adicionar coluna para localizações de captura
    df['capture_location'] = None
        
    # Para cada item da base de dados, resgata as localizações de captura
    for index, row in df.iterrows():
        number = row['id']
        locations = get_capture_locations(number)
        print(f"Localizações de captura para o Pokémon número {number}: {locations}")
        df.at[index, 'capture_location'] = locations
        # time.sleep(1)  # Respeitar o limite de requisições

    # Salvar a base apenas com colunas ID e locations para resgatar e evitar fazer requisição novamente
    df_capture_locations = df[['id', 'capture_location']]
    df_capture_locations.to_json('./outputs/pokemon_capture_locations.json', orient='records', force_ascii=False, indent=4)
    

# Salvar 
df.to_csv('./outputs/pokemon-basic-db.csv', index=False)

print(f"Tamanho do DataFrame atualizado: {df.shape}")
print(df.head())  # Exibir as primeiras linhas do DataFrame atualizado


Dados de localizações de captura carregados do cache.
Tamanho do DataFrame atualizado: (1025, 16)
   id number        name             type  height  weight  attack  defense  \
0   1   0001   Bulbasaur  [grass, poison]     0.7     6.9    49.0     49.0   
1   2   0002     Ivysaur  [grass, poison]     1.0    13.0    62.0     63.0   
2   3   0003    Venusaur  [grass, poison]     2.0   100.0    82.0     83.0   
3   4   0004  Charmander           [fire]     0.6     8.5    52.0     43.0   
4   5   0005  Charmeleon           [fire]     1.1    19.0    64.0     58.0   

     hp  sp_atk  sp_def  speed  total   abilities  \
0  45.0    65.0    65.0   45.0  318.0  [Overgrow]   
1  60.0    80.0    80.0   60.0  405.0  [Overgrow]   
2  80.0   100.0   100.0   80.0  525.0  [Overgrow]   
3  39.0    60.0    50.0   65.0  309.0     [Blaze]   
4  58.0    80.0    65.0   80.0  405.0     [Blaze]   

                       weakness  \
0  [Fire, Ice, Flying, Psychic]   
1  [Fire, Ice, Flying, Psychic]   
2  [Fire,

In [10]:
# One-hot encoding para cada campo de array

# Garante que os campos de array estejam como listas (caso estejam ausentes)
for col in ['abilities', 'type', 'weakness', 'capture_location']:
    df[col] = df[col].apply(lambda x: x if isinstance(x, list) else [])

# Extrair valores únicos de habilidades, tipos, fraquezas e localizações de captura
abilities_dummies = df['abilities'].explode().str.strip().str.lower().dropna().unique()
type_dummies = df['type'].explode().str.strip().str.lower().dropna().unique()
weakness_dummies = df['weakness'].explode().str.strip().str.lower().dropna().unique()
capture_location_dummies = df['capture_location'].explode().str.strip().str.lower().dropna().unique()

# Para cada habilidade, tipo, fraqueza e localização de captura, cria colunas one-hot verificando se o valor está presente
df_abilities = df['abilities'].apply(lambda x: pd.Series({f'ability_{a}': int(a in x) for a in abilities_dummies}))
df_type = df['type'].apply(lambda x: pd.Series({f'type_{t}': int(t in x) for t in type_dummies}))
df_weakness = df['weakness'].apply(lambda x: pd.Series({f'weakness_{w}': int(w in x) for w in weakness_dummies}))
df_capture_location = df['capture_location'].apply(lambda x: pd.Series({f'location_{l}': int(l in x) for l in capture_location_dummies}))

# Concatenar tudo
df_final = pd.concat([
    df.drop(['abilities', 'type', 'weakness', 'capture_location'], axis=1),
    df_abilities,
    df_type,
    df_weakness,
    df_capture_location
], axis=1)

# Salvar em CSV
df_final.to_csv('./outputs/pokedex-encoded-db.csv', index=False, encoding='utf-8')

print("CSV com one-hot encoding gerado com sucesso!")
print(f"Tamanho do DataFrame final: {df_final.shape}\n\n")
print(df_final.head(5))

CSV com one-hot encoding gerado com sucesso!
Tamanho do DataFrame final: (1025, 1189)


   id number        name  height  weight  attack  defense    hp  sp_atk  \
0   1   0001   Bulbasaur     0.7     6.9    49.0     49.0  45.0    65.0   
1   2   0002     Ivysaur     1.0    13.0    62.0     63.0  60.0    80.0   
2   3   0003    Venusaur     2.0   100.0    82.0     83.0  80.0   100.0   
3   4   0004  Charmander     0.6     8.5    52.0     43.0  39.0    60.0   
4   5   0005  Charmeleon     1.1    19.0    64.0     58.0  58.0    80.0   

   sp_def  ...  location_ultra-space-ultra-desert  \
0    65.0  ...                                  0   
1    80.0  ...                                  0   
2   100.0  ...                                  0   
3    50.0  ...                                  0   
4    65.0  ...                                  0   

   location_verdant-cavern--trial-site-area  location_ultra-space-ultra-plant  \
0                                         0                  

## Dicionário de dados